In [ ]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

run_name = "noble-field-7"

In [ ]:
run = wandb.init()
links = {
    "GOOD_AVERAGE_glamorous-sweep-62": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v283",
    "GOOD_azure-sweep-54": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v279",
    "GOOD_apricot-sweep-17": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v242",
    "GOOD_hearty-sweep-60": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v280",
    "GOOD_worldly-sweep-22": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v245",
    "GOOD_legendary-sweep-5": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v230",
    "drawn_river_6": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v2",
    "worldly-firebrand-5": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v1",
    "noble-field-7": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v3"
}
artifact = run.use_artifact(links[run_name], type='model')
artifact_dir = artifact.download()

In [ ]:
import sys
sys.path.insert(0, "../..")

In [ ]:
from helpers import load_variational_mgt_model
from model import GrooveTransformerEncoderVAE
import torch

In [ ]:
from helpers import load_variational_mgt_model   
import os

In [ ]:
model = load_variational_mgt_model(os.path.join(artifact_dir, "100.pth"))

In [ ]:
from data import load_gmd_hvo_sequences

train_set = load_gmd_hvo_sequences(
    dataset_setting_json_path = "../../data/dataset_json_settings/4_4_Beats_gmd.json", 
    subset_tag = "test", 
    force_regenerate=False)

In [ ]:
gt_sample = train_set[0]
groove = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)
gt_sample.metadata

In [ ]:
mu, logvar = model.encode_to_mu_logvar(groove)
latent_z = model.reparametrize(mu, logvar)

In [ ]:
latent_z

In [ ]:
voice_thresholds = [0.5] * 9
voice_max_count_allowed = [32] * 9
h, v, o = model.sample(latent_z=latent_z,
                       voice_thresholds=voice_thresholds,
                       voice_max_count_allowed=voice_max_count_allowed,
                       return_concatenated=False,
                       sampling_mode=0)
print(h, v, o)

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd


In [14]:
latents = []
labels = []
metadatas = []
use_all_styles = True

for gt_sample in train_set:
    if (use_all_styles or (gt_sample.metadata["style_primary"] in ["rock", "funk", "afrobeat"])):
        metadatas.append(gt_sample.metadata)
        labels.append(gt_sample.metadata["style_primary"])
        groove = torch.zeros((1, 32, 3))
        flattened_ = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)[:,:32, :]
        t_steps = flattened_.shape[1]
        groove[:, :t_steps, :] = flattened_
        mu, logvar = model.encode_to_mu_logvar(groove)
        latent_z = model.reparametrize(mu, logvar)
        latents.append(latent_z.detach().cpu().numpy())

latents = np.array(latents).squeeze(1)
features = np.expand_dims(latents, -1) # we use each dimension of latent_z as a feature
feature_labels = [f"z_{dim}" for dim in range(features.shape[1])]

In [15]:
import pandas as pd
data = {"style_primary": [style for style in labels]}
data.update({f"z_{dim_i}": latents[:, dim_i] for dim_i in range(features.shape[1])})
df = pd.DataFrame(data)
df.head()

,style_primary,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8,...,z_118,z_119,z_120,z_121,z_122,z_123,z_124,z_125,z_126,z_127
0,soul,1.103464,-0.626700,-0.198182,-0.475626,0.903612,1.176979,0.007071,-1.041442,-0.532061,...,-1.407513,-1.766600,0.244445,-0.343117,0.493831,0.194009,-0.491197,0.833408,1.829738,-1.031805
1,soul,-0.128921,-1.169329,-0.530025,1.133850,-0.088824,-0.676383,-0.968475,0.684501,0.308862,...,-0.204265,-0.836512,1.142790,2.095299,-0.865173,1.231519,-0.755949,1.260754,1.949624,-0.265518
2,soul,-1.628883,-1.120446,-0.006517,0.367408,0.551159,1.970728,-0.098604,-0.393517,-0.466214,...,0.484740,-0.091725,1.484421,0.078342,1.401415,0.500967,-0.233326,1.772177,1.666774,-1.516757
3,soul,0.424331,-0.720564,0.318271,1.695400,0.329941,1.417686,-0.688277,-0.670036,-0.126297,...,-0.171639,0.380275,1.566107,-0.192974,0.535072,1.059456,-0.539561,1.127323,1.365738,0.399845
4,soul,0.503676,-1.312784,-1.121984,1.591133,-0.169920,0.575592,0.883954,-0.198365,-1.374069,...,-0.078318,-1.509660,3.626695,1.878944,1.846746,-0.214075,-1.364839,1.611233,1.570689,1.026336


In [16]:
from umap import UMAP

In [17]:
embedding_dims = 2
umap = UMAP(n_components=embedding_dims, metric="euclidean", n_neighbors=20)
embeddings = umap.fit_transform(df.drop("style_primary", axis=1))

In [18]:
from bokeh.palettes import inferno, Category20b
from bokeh.core.enums import MarkerType

styles = sorted(df.style_primary.unique())
hues = inferno(len(styles))
hues = Category20b[20]
style_hue_map = {f"{style}": hues[i] for i, style in enumerate(styles)}

bases = [
    "circle",
    "diamond",
    "inverted_triangle",
    "square",
    "star",
    "triangle",
] * 4

marker_map = {style: bases[ix] for ix, style in enumerate(styles)}

In [19]:
from collections import OrderedDict
from bokeh.plotting import figure, show, save

embedding_dict = dict()
styles = df.style_primary


for ix, style in enumerate(styles):
    if(not style in embedding_dict):
        embedding_dict.update({style: ([], [])})
    embedding_dict[style][0].append(embeddings[ix][0])
    embedding_dict[style][1].append(embeddings[ix][1])
    

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

for key, values in embedding_dict.items():
    p.scatter(values[0], values[1],
              fill_color=style_hue_map[key], 
              line_color=None, legend_label=key, marker=marker_map[key], size=12)
p.legend.click_policy="hide"
save(p, f"UMAP_{run_name}.html")

'/Users/behzadhaki/Github/GrooveTransformer/eval/UMAP/UMAP_noble-field-7.html'

In [ ]:
embedding_dict

In [ ]:
# from sklearn.manifold import TSNE
# umap = TSNE(n_components=embedding_dims)
# embeddings = umap.fit_transform(df.drop("style_primary", axis=1))

In [ ]:
# import numpy as np

# from bokeh.plotting import figure, show, save
# from bokeh.io import output_notebook

# TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# p = figure(tools=TOOLS)

# p.scatter(embeddings[:, 0], embeddings[:, 1],
         #  fill_color=colors, fill_alpha=0.6,
          # line_color=None)

# save(p, "TSNE.html") 